# Coursera Capstone Project - Week3 - Activity 3

In [1]:
#Importing libraries 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Importing Geospatial_Coordinates csv to dataframe
The dataframe will consist of three columns: 'Postal Code','Latitude','Longitude'

In [2]:
url = 'https://github.com/steffim88/Coursera_Capstone/blob/master/Geospatial_Coordinates.csv'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])


df_geo = pd.DataFrame(data, columns=['Index','Postal Code','Latitude','Longitude'])
df_geo = df_geo[['Postal Code','Latitude','Longitude']]
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.8066863,-79.1943534
1,M1C,43.7845351,-79.1604971
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.2169174
4,M1H,43.773136,-79.2394761


# Importing Canada csv to dataframe

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
df.dropna(inplace = True)
df = df[(df.Borough != 'Not assigned') ]
def list_neighborhood(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
df_final = df.groupby(['PostalCode', 'Borough']).apply(list_neighborhood).reset_index(name='Neighborhood')
df_final.loc[df_final['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_final['Borough']
df_final.head()



,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Join 2 dataframes

In [4]:
df_new = pd.concat([df_final,df_geo ] ,axis=1)
df_new.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",M1C,43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,M1G,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.2394761


## Explore and cluster the neighborhoods in Torondo

In [5]:
df_new = df_new[ df_new.Borough.str.contains('Toronto') ]
df_new.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
37,M4E,East Toronto,The Beaches,M4E,43.6763574,-79.2930312
41,M4K,East Toronto,"Riverdale, The Danforth West",M4K,43.6795571,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",M4L,43.6689985,-79.3155716
43,M4M,East Toronto,Studio District,M4M,43.6595255,-79.340923
44,M4N,Central Toronto,Lawrence Park,M4N,43.7280205,-79.3887901


In [6]:
#import libraries
!pip install folium 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

Plot each data on the map

In [7]:

toronto_coords = [43.653226, -79.3831843]

In [8]:
map_toronto = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=12)

In [9]:
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
  

In [10]:
  map_toronto

explore the first neighborhood in our dataframe

In [11]:
toronto_data = pd.DataFrame(df_new)
toronto_data = toronto_data.reset_index().drop('index', axis=1)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M4E,East Toronto,The Beaches,M4E,43.6763574,-79.2930312
1,M4K,East Toronto,"Riverdale, The Danforth West",M4K,43.6795571,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",M4L,43.6689985,-79.3155716
3,M4M,East Toronto,Studio District,M4M,43.6595255,-79.340923
4,M4N,Central Toronto,Lawrence Park,M4N,43.7280205,-79.3887901


Get first neighbourhoods Latitude and Longitude value

In [12]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] 
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] 
neighborhood_name = toronto_data.loc[0, 'Neighborhood']

get the top 100 venues that are in East Toronto within a radius of 500 meters

In [13]:
VERSION = '20180605'
CLIENT_ID = 'JWUMXPYGSPA2V4NOTS2RYPEDTDD1XCGZ5RIMK3HWGP3HMZI4'
CLIENT_SECRET = 'F2XQYTAHZCEK2M3UDVQXFMKZGJABJG4ZM0ZHBFBKO223H3IZ'
latitude = neighborhood_latitude
longitude = neighborhood_longitude
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,radius, LIMIT)

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d5e4a1fa6ec98002c3c6405'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.6808574045, 'lng': -79.28682091449052},
   'sw': {'lat': 43.6718573955, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distance': 89,
     

get_category_type function from the Foursquare lab

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
from pandas.io.json import json_normalize 
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


Explore Neighborhoods in Toronto

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
       
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:

toronto_venues = getNearbyVenues(toronto_data['Neighborhood'], toronto_data['Latitude'], toronto_data['Longitude'])

The Beaches
Riverdale, The Danforth West
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [18]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


# Analyze Each Neighborhood

In [19]:

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.00,0.000000,0.00000,0.00,0.010000,0.000000,0.000000,0.000000,0.010000,0.01
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.00
3,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Business Reply Mail Processing Centre 969 Eastern,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,...,0.00,0.000000,0.00000,0.00,0.011905,0.000000,0.011905,0.000000,0.011905,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.01,0.000000,0.00000,0.00,0.060000,0.000000,0.000000,0.040000,0.010000,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.011905,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,...,0.00,0.000000,0.00000,0.00,0.000000,0.011905,0.000000,0.011905,0.000000,0.00


print each neighborhood along with the top 10 most common venues

In [21]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print(hood)
    print('---------------------------------------------------------------------------------------------------------------------------------------------')
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    

Adelaide, King, Richmond
---------------------------------------------------------------------------------------------------------------------------------------------
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2           Steakhouse  0.04
3                  Bar  0.04
4      Thai Restaurant  0.04
5     Asian Restaurant  0.03
6                  Gym  0.03
7       Breakfast Spot  0.03
8  American Restaurant  0.03
9                Hotel  0.03
Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara
---------------------------------------------------------------------------------------------------------------------------------------------
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14
2    Airport Terminal  0.14
3         Coffee Shop  0.07
4     Harbor / Marina  0.07
5             Airport  0.07
6  Airport Food Court  0.07
7        Airport Gate  0.07
8    Sculptur

                     venue  freq
0                     Café  0.11
1                      Bar  0.06
2               Restaurant  0.06
3      Japanese Restaurant  0.06
4                   Bakery  0.06
5                Bookstore  0.06
6              College Gym  0.03
7  Comfort Food Restaurant  0.03
8                 Beer Bar  0.03
9               Beer Store  0.03
Harbourfront East, Toronto Islands, Union Station
---------------------------------------------------------------------------------------------------------------------------------------------
                 venue  freq
0          Coffee Shop  0.11
1                Hotel  0.05
2             Aquarium  0.05
3   Italian Restaurant  0.04
4                 Café  0.04
5  Sporting Goods Shop  0.03
6              Brewery  0.03
7       Scenic Lookout  0.03
8          Pizza Place  0.03
9  Fried Chicken Joint  0.03
Harbourfront, Regent Park
----------------------------------------------------------------------------------------------------

                 venue  freq
0                Trail   0.2
1    Health Food Store   0.2
2          Coffee Shop   0.2
3                  Pub   0.2
4  Monument / Landmark   0.0
5          Music Venue   0.0
6          Music Store   0.0
7               Museum   0.0
8        Movie Theater   0.0
9          Yoga Studio   0.0


In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
import numpy as np
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",Airport Service,Airport Terminal,Airport Lounge,Plane,Sculpture Garden
2,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse
3,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Furniture / Home Store,Intersection
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden
5,"Cabbagetown, St. James Town",Park,Coffee Shop,Café,Pizza Place,Convenience Store
6,Central Bay Street,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bakery,Vietnamese Restaurant
8,Christie,Café,Grocery Store,Park,Diner,Convenience Store
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant


# Cluster Neighborhood

In [24]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]
toronto_merged = toronto_data

toronto_merged['Cluster Labels'] = kmeans.labels_

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,M4E,43.6763574,-79.2930312,0,Health Food Store,Trail,Pub,Coffee Shop,Farmers Market
1,M4K,East Toronto,"Riverdale, The Danforth West",M4K,43.6795571,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store
2,M4L,East Toronto,"India Bazaar, The Beaches West",M4L,43.6689985,-79.3155716,0,Pizza Place,Fast Food Restaurant,Italian Restaurant,Pet Store,Gym
3,M4M,East Toronto,Studio District,M4M,43.6595255,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant
4,M4N,Central Toronto,Lawrence Park,M4N,43.7280205,-79.3887901,0,Park,Bus Line,Dim Sum Restaurant,Swim School,Women's Store


# Plot Map

In [25]:

map_clusters = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=12)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,-79.2930312,0,Health Food Store,Trail,Pub,Coffee Shop,Farmers Market
1,East Toronto,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store
2,East Toronto,-79.3155716,0,Pizza Place,Fast Food Restaurant,Italian Restaurant,Pet Store,Gym
3,East Toronto,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant
4,Central Toronto,-79.3887901,0,Park,Bus Line,Dim Sum Restaurant,Swim School,Women's Store
5,Central Toronto,-79.3901975,0,Breakfast Spot,Hotel,Sandwich Place,Clothing Store,Food & Drink Shop
6,Central Toronto,-79.4056784,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Diner
7,Central Toronto,-79.3887901,0,Sandwich Place,Dessert Shop,Pizza Place,Café,Restaurant
8,Central Toronto,-79.3831599,0,Playground,Gym,Restaurant,Park,Women's Store
9,Central Toronto,-79.4000493,0,Coffee Shop,Pub,Pizza Place,Sushi Restaurant,Fried Chicken Joint


# Cluster 2

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
26,Downtown Toronto,-79.4000493,1,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bakery,Vietnamese Restaurant
31,West Toronto,-79.4422593,1,Bakery,Pharmacy,Supermarket,Pizza Place,Coffee Shop


# Cluster 3

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
32,West Toronto,-79.4197497,2,Bar,Coffee Shop,Asian Restaurant,Cocktail Bar,Boutique


# Cluster 4

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,Downtown Toronto,-79.3873826,3,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint


# Cluster 4

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
24,Central Toronto,-79.4056784,4,Coffee Shop,Sandwich Place,Café,Pizza Place,BBQ Joint
